<a href="https://colab.research.google.com/github/park1125/Curating_Code/blob/master/Enphase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

태양광 관련 웹사이트, Enphase Website에서 태양광 발전기가 만들어낸 에너지량과 관련된 데이터를 긁어오는 것이 목적입니다. 2014년부터 현재까지 5분마다 생성된 에너지 양이 powr column 밑에 적히고, 시간의 단위는 unix epoch 입니다 (5분 = 300). 24시간 내내 에너지가 만들어지는 것이 아니고, 해가 떠 있을 때만 에너지가 만들어지기 때문에, 해가 지고 난 후에는 에너지가 기록 된 바가 없습니다.

- 문제점:
1. 일단 데이터 양이 좀 많아서 시간이 오래걸린다. 빠르고 효율적으로 데이터를 받을 수는 없을까?
***2. 일단 원본 데이터는 unix epoch 시간 단위여서 나중에 일반 시간 단위로 다 바꿔야 한다. - 해결***

***3. 가장 큰 문제점은, 데이터가 프린트 되다가 끊긴다. 그래서 한번도 성공적으로 마무리를 한 적이 없다. (1576097400 뒤에는 맨끝에는 000이 적힌다. 그 말은, 데이터가 없다는 말인데, 데이터가 없을 리가 없다...(?) (사실 아직 확인을 안해봤다.) -> 무료계정이여서 그렇다.. 이건 교수님이 알아서 하셔야 할 문제이다...***

**4. excel로 전환뒤에 어디다가 저장하는지 안적어놨다... - 바탕화면 경로 받아서 바탕화면 밑에 result 라는 폴더 생성 후 result.xlxs 라고 파일 만들게 바꾸고 해결**
5. 약 6년치의 데이터를 한번에 받아내야 하는 이유는...? 
6. 혹시 5분마다 받지 말고 그냥 더해서 한시간단위의 에너지를 받을 수는 없을까..?


In [ ]:
#!/usr/bin/python3.8
# -*- coding:utf-8 -*-

# This code is for both web-crawling and web-parsing! (Modified)
# Using Python 3.8, You Need PIP install pandas, requests, xlsxwriter
import requests, os
import pandas as pd
import numpy as np
import time, sys

def get_syspath():
    import platform

    if("Windows" == platform.system()):
        return str("C:\\Users\\%s\\Desktop\\" %str(os.getlogin()))
    else:
        return str(os.path.join(os.path.expanduser('~'), "Desktop"))

syspath = get_syspath()
result_path = os.path.join(syspath, "result")
try:
    os.mkdir(result_path)
except:
    pass
result_file = os.path.join(result_path, "result.xlsx")


# status code for calling API :
queryString = {'user_id': '4d6a41304e4459774e773d3d0a', 'key': '4b7970e0372e37e2fd96cce6cb8d1578'}


# --------- Loop everyday until it reaches the date of latest report --------- #
# DateOfStart Input value is Only Int Unix Time Value..
DateOfStart = 1398337800 #1398337800 time value string is "Thu Apr 24 20:10:00 2014"
counter = 1
xlsx_writer = pd.ExcelWriter(result_file, engine='xlsxwriter')

while DateOfStart < 1576097700:
    # ------ trying to get 'stats' (give powr and enWh every 5 min) ------ #
    response = requests.get(
        'https://api.enphaseenergy.com/api/v2/systems/351546/stats?start_at=' + str(DateOfStart) + '&end_at=' + str(time.time()), params = 
        queryString)

    data = response.json()
        
    if 'intervals' in data: # not if data['intervals']
        df = pd.DataFrame(data['intervals'])
        if df.empty:
            print('API Limit Detect!!\nPlease Upgrade Plan')
            break
        else:
            time_data = []
            for index, data in df.iterrows():
                time_data.append(time.ctime(df.iloc[index]["end_at"]) + " - " + time.ctime(df.iloc[index]["end_at"]+299))
            df["end_at"] = time_data
            df = df.set_index("end_at")
            
    else:
        print('API Limit Detect!!\nPlease Upgrade Plan')
        break

    # --------- try to save append it to the excel file! --------- #
    df.to_excel(xlsx_writer,sheet_name = str(counter) ,startrow = 0, startcol = 0)
    counter +=1
    DateOfStart += 86400
    # is this right? why increase 86400? 86400 second to 1 Day
    # Delay setting to Protect ban from server
    time.sleep(10)


xlsx_writer.save()